In [14]:
import nibabel as nib
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from zipfile import ZipFile

In [15]:
# Constants
NB=False
def my_print(msg):
    if NB:
        print(msg)
      

def generate_np_for_resizing(ficheiro_scan):
    num_imagem=0
    image_list = []
    scan = nib.load(ficheiro_scan)
    scan = np.flip(scan.get_fdata().T)
    snp=np.asarray(scan)
    #normalize scan
    snp = np.floor(snp)
    snp /= np.max(snp)  
    for img in snp :
        image_list.append(img)
    image_list = np.array(image_list)
    image_list = image_list.reshape(image_list.shape[0],image_list.shape[1],image_list.shape[2],1)
    return image_list


def calc_i_f(n,med,nbb):
    l_half=nbb//2
    r_half=nbb-l_half
    if med-l_half<0:
        l_half=med
        r_half=nbb-l_half
    elif med+r_half>n:
        r_half = n-med
        l_half = nbb-r_half
    vi = med - l_half
    vf = med + r_half
    return int(vi), int(vf)

BB_IMAGE_HEIGHT = 32 
BB_IMAGE_WIDTH = 32   

def process_centers(model_r, scan):
    n =scan.shape[0]
    scan_w = scan.shape[1]
    scan_h = scan.shape[2]
    xp_sum=yp_sum=slicep_sum = 0
    for i in range(n):
        im = scan[i]
        im2 = im.reshape(1,im.shape[0],im.shape[1],im.shape[2])
        p=model_r.predict(im2)
        xp,yp = p[0]
        xp_sum+=xp
        yp_sum+=yp
    xp_med = xp_sum//n
    yp_med = yp_sum//n
    xi,xf = calc_i_f(scan_w,xp_med,BB_IMAGE_WIDTH)
    yi,yf = calc_i_f(scan_h,yp_med,BB_IMAGE_HEIGHT) 
    return  xi,xf,yi,yf 
    

def process_bulnobulb(model_c, scan):
    folga=4
    n =scan.shape[0]
    slicep_sum = 0
    qtp_with_bulbo = 0
    for i in range(n):
        im = scan[i]
        im2 = im.reshape(1,im.shape[0],im.shape[1],im.shape[2])
        scan_has_bulb = model_c.predict(im2) 
        print("scan_has_bulb: ", scan_has_bulb)
        if scan_has_bulb[0] >0.2:
            qtp_with_bulbo+=1
            slicep_sum+=i  
    if qtp_with_bulbo<=1:
        slicep_med = 0
        si=sf=0
    else:
        slicep_med = slicep_sum//qtp_with_bulbo
        if qtp_with_bulbo+folga > n:
            profundidade=n
        else:
            profundidade=qtp_with_bulbo+folga
        si,sf = calc_i_f(n,slicep_med,profundidade)
    return si, sf
    

def generate_BB_scan(model_c, model_r, scan):
    si,sf = process_bulnobulb(model_c, scan)
    if si==sf :
        my_print("Não Tem Bulb")
        return None
    else:
        xi,xf,yi,yf = process_centers(model_r, scan[si:sf,:,:])  
    return scan[si:sf,xi:xf,yi:yf], (si,sf,xi,xf,yi,yf)

def get_affine(ficheiro_scan):
    scan = nib.load(ficheiro_scan)
    mask_affine=scan.affine
    mask_header=scan.header
    return mask_affine,mask_header

def predict_pos_processing(mask,threshold):
    return np.where(mask > threshold, 1., 0.)


def generate_mask(model_s,scan_BB, shape_scan,coord_BB):
    (si,sf,xi,xf,yi,yf) = coord_BB
    masks=np.zeros(shape_scan)
    mask_BB=np.zeros(scan_BB.shape)
    n =scan_BB.shape[0]
    for i in range(n):
        im = scan_BB[i]
        im2 = im.reshape(1,im.shape[0],im.shape[1],im.shape[2])
        pred = predict_pos_processing(model_s.predict(im2), 0.7)
        mask_BB[i]=pred[0]
    masks[si:sf,xi:xf,yi:yf]=mask_BB
    return masks


def save_mask(ficheiro_scan, masks, file_name, output_path):
    mask_affine, mask_header=get_affine(ficheiro_scan)
    my_print(masks.shape)
    masks = masks.reshape(masks.shape[0],masks.shape[1],masks.shape[2])
    mask = np.flip(masks).T
    my_print(mask.shape)
    seg_mask = nib.Nifti1Image(mask, mask_affine, mask_header)
    nib.save(seg_mask, output_path + file_name )


In [24]:
## QUANDO TENTO FAZER LOAD DOS MODELS ASSIM DÁ ERRO 
## A CORRER O RUN DA ULTIMA CELULA DESTE NOTEBOOK

import tempfile
import h5py

modelpaths = [
    {
        "name": "modelo_CNN_R_treinado_v4_174x190.h5",
        "path": "./modelfiles/modelo_CNN_R_treinado_v4_174x190.h5"
    },
    {
        "name": "modelo_CNN_treinado_v4.h5",
        "path": "./modelfiles/modelo_CNN_treinado_v4.h5"
    },
    {
        "name": "modelo_FCN_segmentation_32x32.h5",
        "path": "./modelfiles/modelo_FCN_segmentation_32x32.h5"
    }
    ]

def load_models(modelpaths):
    global model_c
    global model_r
    global model_s
    
    with tf.device('/cpu:0'):
        for i in modelpaths:
            name = i["name"]
            path = i["path"]
            if (name == "modelo_CNN_treinado_v4.h5"):
                model_c = load_model(path)
                model_c.summary()
            if (name == "modelo_CNN_R_treinado_v4_174x190.h5"):
                model_r = load_model(path)
                model_r.summary()
            if (name == "modelo_FCN_segmentation_32x32.h5"):
                model_s = load_model(path, compile=False)
                model_s.summary()
    



print("LOAD MODELS:", load_models(modelpaths))




Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 174, 190, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 172, 188, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 170, 186, 32)      9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 170, 186, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 85, 93, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 83, 91, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 81, 89, 64)        36928 

In [29]:
def iterate(lst, start, end):
    if start < 0 or end >= len(lst) or start > end:
        return print(lst[start])
    iterate(lst, start + 1, end)

iterate([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 2, 8)

10


In [3]:
##RECURSIVIDADE DA ERRO

modelpaths = [
    {
        "name": "modelo_CNN_R_treinado_v4_174x190.h5",
        "path": "./modelfiles/modelo_CNN_R_treinado_v4_174x190.h5"
    },
    {
        "name": "modelo_CNN_treinado_v4.h5",
        "path": "./modelfiles/modelo_CNN_treinado_v4.h5"
    },
    {
        "name": "modelo_FCN_segmentation_32x32.h5",
        "path": "./modelfiles/modelo_FCN_segmentation_32x32.h5"
    }
    ]

def printElement(inputlist):
    global model_c
    global model_r
    global model_s
    newlist=inputlist
    if len(newlist)==0:
        return
    else:
        removedElement=newlist[len(inputlist)-1]
        newlist=newlist[:len(inputlist)-1]
        Element=printElement(newlist)
        name = removedElement["name"]
        path  = removedElement["path"]
        if (name == "modelo_CNN_R_treinado_v4_174x190.h5"):
            model_c = load_model(f"{path}")
        if (name == "modelo_CNN_treinado_v4.h5"):
            model_r = load_model(f"{path}")
        if (name == "modelo_FCN_segmentation_32x32.h5"):
            model_s = load_model(f"{path}", compile=False)


printElement(modelpaths)

In [3]:
## ASSIM NAO DA ERRO A CORRER O RUN DA ULTIMA CELULA DESTE NOTEBOOK

def load_models2():
    global model_c
    global model_r
    global model_s
    try:
        with tf.device('/cpu:0'):
            #model to classify images with bulb or without bulb
            model_c = load_model("./modelfiles/modelo_CNN_treinado_v4.h5") 
            print("model_c: ",  model_c)
            #model to calculate centers of images with bulb                  
            model_r = load_model("./modelfiles/modelo_CNN_R_treinado_v4_174x190.h5")
            print("model_r: ",  model_r)
            #segmentation model
            model_s = load_model("./modelfiles/modelo_FCN_segmentation_32x32.h5" , compile=False)
            print("model_s: ",  model_s)
            print("models lodaded successfully!")
        return True
    except: 
        return False

print("LOAD MODELS2:", load_models2())

LOAD MODELS2: False


In [ ]:
print(model_c)
print(model_r)

In [19]:
##UNICO QUE FUNCIONA
with tf.device('/cpu:0'):
        #model to classify images with bulb or without bulb
        model_c = load_model("./modelfiles/modelo_CNN_treinado_v4.h5") 
        model_c.summary()
        #model to calculate centers of images with bulb                  
        model_r = load_model("./modelfiles/modelo_CNN_R_treinado_v4_174x190.h5")
        model_r.summary()
        #segmentation model
        model_s = load_model("./modelfiles/modelo_FCN_segmentation_32x32.h5" , compile=False)
        model_s.summary()
        print("models lodaded successfully!")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 174, 190, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 172, 188, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 170, 186, 32)      9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 170, 186, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 85, 93, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 83, 91, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 81, 89, 64)        36928 

In [3]:
modelpaths = {
        "modelo_CNN_R_treinado_v4_174x190.h5": "./modelfiles/modelo_CNN_R_treinado_v4_174x190.h5",
        "modelo_CNN_treinado_v4.h5": "./modelfiles/modelo_CNN_treinado_v4.h5",
        "modelo_FCN_segmentation_32x32.h5": "./modelfiles/modelo_FCN_segmentation_32x32.h5",
    }



def load_final(modelpaths):
    global path1
    global path2
    global path3
    path1=modelpaths["modelo_CNN_R_treinado_v4_174x190.h5"]
    path2=modelpaths["modelo_CNN_treinado_v4.h5"]
    path3=modelpaths["modelo_FCN_segmentation_32x32.h5"]

load_final(modelpaths)

print(type(path1))
    
with tf.device('/cpu:0'):
    #model to classify images with bulb or without bulb
    model_c = load_model(path1) 
    #model to calculate centers of images with bulb                  
    model_r = load_model(path2)
    #segmentation model
    model_s = load_model(path3 , compile=False)






<class 'str'>


NameError: name 'tf' is not defined

In [5]:
print(model_c, model_r, model_s)

In [25]:
input_file_name = "00912_006.nii.gz"
input_file_path = "./input/00912_006.nii.gz"

output_directory_path = "./output/"
output_file_name = "00912_006_masks.nii.gz"

def run(input_file_path, output_file_name, output_directory_path):
    np_original_all_scan = generate_np_for_resizing(input_file_path)
    scan_BB, coord_BB = generate_BB_scan(model_c,model_r,np_original_all_scan)
    shape_scan = np_original_all_scan.shape
    masks = generate_mask(model_s,scan_BB, shape_scan, coord_BB)  
    save_mask(input_file_path, masks, output_file_name, output_directory_path)

print(run(input_file_path, output_file_name, output_directory_path))

scan_has_bulb:  [[0.57968956]]
scan_has_bulb:  [[0.00747663]]
scan_has_bulb:  [[0.08426076]]
scan_has_bulb:  [[0.0061]]
scan_has_bulb:  [[0.00011863]]
scan_has_bulb:  [[2.2316652e-05]]
scan_has_bulb:  [[0.01286075]]
scan_has_bulb:  [[0.22620669]]
scan_has_bulb:  [[0.5443514]]
scan_has_bulb:  [[0.93271756]]
scan_has_bulb:  [[0.980564]]
scan_has_bulb:  [[0.9894799]]
scan_has_bulb:  [[0.7446953]]
scan_has_bulb:  [[0.94629705]]
scan_has_bulb:  [[0.77954644]]
scan_has_bulb:  [[0.97250134]]
scan_has_bulb:  [[0.79150367]]
scan_has_bulb:  [[0.135631]]
scan_has_bulb:  [[0.01063433]]
scan_has_bulb:  [[0.00098851]]
scan_has_bulb:  [[2.582132e-07]]
scan_has_bulb:  [[1.6312703e-11]]
scan_has_bulb:  [[2.6293478e-15]]
scan_has_bulb:  [[3.5187381e-15]]
scan_has_bulb:  [[2.9480484e-13]]
scan_has_bulb:  [[8.215575e-18]]
scan_has_bulb:  [[1.2587313e-19]]
scan_has_bulb:  [[5.358499e-23]]
scan_has_bulb:  [[1.241822e-22]]
scan_has_bulb:  [[3.1899012e-27]]
None


In [8]:
path = "./modelfiles/modelo_CNN_R_treinado_v4_174x190.h5"

model_1 = load_model(path)

model_2 = load_model("./modelfiles/modelo_CNN_R_treinado_v4_174x190.h5")

print(model_1==model_2)

False


In [10]:
model_1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 174, 190, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 172, 188, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 170, 186, 32)      9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 170, 186, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 85, 93, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 83, 91, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 81, 89, 64)        36928 

In [11]:
model_2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 174, 190, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 172, 188, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 170, 186, 32)      9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 170, 186, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 85, 93, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 83, 91, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 81, 89, 64)        36928 